In [ ]:
import torch
print(torch.cuda.is_available() , torch.__version__)

In [1]:
from src2.api import ModelTrainer  
app = ModelTrainer.initialize(stage = 0 , resume = 0 , checkname= 1)
app.go()

24-09-17 21:10:04|MOD:display     |: Model Specifics:
24-09-17 21:10:04|MOD:display     |: Start Process [Data] at Tue Sep 17 21:10:04 2024!


--Process Queue : Data + Fit + Test
--Start Training New!
--Model_name is set to gru_day_ShortTest!
ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
CallbackTimer(verbosity=10) , record time cost of callback hooks
EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
BatchDisplay(verbosity=10) , display batch progress bar
StatusDisplay(verbosity=10) , display epoch and event information
DetailedAlphaAnalysis(use_num=avg) , record and concat each model to Alpha model instance
{'model_name': 'gru_day_ShortTest',
 'model_module': 'gru',
 'model_submodels': ['best', 'swalast', 'swabest'],
 'model_boost

24-09-17 21:10:06|MOD:display     |: Finish Process [Data], Cost 2.3 Secs
24-09-17 21:10:06|MOD:display     |: Start Process [Fit] at Tue Sep 17 21:10:06 2024!
24-09-17 21:10:06|MOD:classes     |: First Iterance: (20170103 , 0)


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


Train Ep#  0 loss : 1.01279:   3%|▎         | 1/38 [00:00<00:09,  3.78it/s]

...

Grouped Return Results are saved to d:\Coding\learndl\learndl\models\gru_day_ShortTest\detailed_analysis\group.xlsx


,factor_name,benchmark,start,end,pf,bm,excess,annualized,mdd,te,ir,calmar,turnover,mdd_period
1,best,csi300,20170104,20170302,1.10%,1.97%,-0.87%,-5.41%,1.63%,4.46%,-1.214,-3.314,3.400,20170104-20170118
2,best,csi500,20170104,20170302,-1.86%,0.66%,-2.52%,-14.65%,3.01%,5.21%,-2.811,-4.863,3.400,20170104-20170222
0,best,csi1000,20170104,20170302,-1.40%,-2.29%,0.89%,6.32%,1.32%,5.29%,1.196,4.788,3.400,20170104-20170118
4,swabest,csi300,20170104,20170302,1.14%,1.97%,-0.83%,-5.19%,1.84%,4.80%,-1.081,-2.814,3.400,20170104-20170118
5,swabest,csi500,20170104,20170302,-1.38%,0.66%,-2.03%,-12.04%,2.13%,4.38%,-2.748,-5.643,3.400,20170104-20170118
3,swabest,csi1000,20170104,20170302,-2.25%,-2.29%,0.04%,0.35%,1.29%,4.42%,0.079,0.271,3.400,20170104-20170111
7,swalast,csi300,20170104,20170302,1.10%,1.97%,-0.87%,-5.41%,1.63%,4.46%,-1.214,-3.315,3.400,20170104-20170118
8,swalast,csi500,20170104,20170302,-1.86%,0.66%,-2.52%,-14.65%,3.01%,5.21%,-2.811,-4.863,3.400,20170104-20170222
6,swalast,csi1000,20170104,20170302,-1.40%,-2.29%,0.89%,6.32%,1.32%,5.29%,1.196,4.788,3.400,20170104-20170118


Analytic datas are saved to d:\Coding\learndl\learndl\models\gru_day_ShortTest\detailed_analysis\data.xlsx
Analytic plots are saved to d:\Coding\learndl\learndl\models\gru_day_ShortTest\detailed_analysis\plot.pdf
Callback Time costs


,hook_name,num_calls,total_time,avg_time
10,on_fit_model_end,1,9.138,9.138e+00
11,on_fit_model_start,1,0.707,7.075e-01
18,on_test_model_start,1,0.659,6.589e-01
28,on_validation_epoch_end,4,0.075,1.880e-02
15,on_test_batch_start,105,0.016,1.547e-04


24-09-17 21:12:17|MOD:time        |: Main Process Finished! Cost 2 Minutes 13.4 Seconds


In [3]:
from src2.model.callback.test import PRED_RECORD
PRED_RECORD.all_preds[(PRED_RECORD.all_preds['submodel'] == 'best') & (PRED_RECORD.all_preds['secid'] == 688981)]

,model_num,submodel,secid,date,values


In [5]:
from src2.api import ModelHiddenExtractor
extractor = ModelHiddenExtractor('gru_day' , model_nums=[0] , model_submodels=['best'])
extractor.extract_hidden('update' , deploy = True)

AssertionError: short_test

In [ ]:
from src2.api import DataAPI , ModelTrainer
DataAPI.reconstruct_train_data()
ModelTrainer.update_models()

In [ ]:
from src.api import HiddenExtractor
extractor = HiddenExtractor('resnet_gru_30m' , model_nums=[1] , model_types=['best'])
extractor.extract_hidden('update' , deploy = True)

In [ ]:
from src.api import DataAPI , Trainer
DataAPI.reconstruct_train_data()
Trainer.update_models()